In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
import os
import json
#add .env variable to environment
load_dotenv()

#!pip install python-dotenv
# defaults to getting the key using os.environ.get("OPENAI_API_KEY")
# if you saved the key under a different environment variable name, you can do something like:
# client = OpenAI(
#   api_key=os.environ.get("CUSTOM_ENV_NAME"),
# )

True

In [3]:
API_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(
  api_key=API_key,
)


user – Lets the model know that the following content is from the user – ie, the person who asked a question.

assistant – Lets the model know the content was generated as a response to a user.

system – A message specified by the developer to “steer” the model response.  Depending on the model being used, the system message may have more or less impact on the actual model responses.

In [4]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
print(completion.choices[0].message.content)

In the world of code, a method hides,
Whose magic sparkles bright like fireflies.
It's called recursion, a dance of time,
A loop that sings a lovely rhyme.

A function that calls itself, so brave,
An endless loop, a recursive wave.
Like a mirror reflecting its own face,
Recursion goes from place to place.

With each call, a new path unfurled,
A story told to span the world.
A journey deep into the code,
Where loops and branches freely rode.

Through forests dense with data's gleam,
Recursion flows like a sparkling stream.
It dives into the depths of night,
Guided by the programmer's light.

So let us cherish this wondrous art,
This loop that sings straight from the heart.
Recursion, a tale of infinite grace,
In the enchanted realm of coding's space.


In [6]:
recipe = pd.read_csv("cookbook_recipes_nlg_10k.csv")
recipe.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [7]:
training, testing = train_test_split(recipe, test_size = 0.33, random_state=42)


In [8]:
system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."

def create_user_message(row):
    return f"""Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: """

def prepare_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["NER"]})

    return {"messages": messages}

pprint(prepare_conversation(training.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the generic ingredients from each of the '
                          'recipes provided.',
               'role': 'system'},
              {'content': 'Title: Christmas Morning Sticky Buns\n'
                          '\n'
                          'Ingredients: ["1/2 Cup Chopped Pecans or Walnuts", '
                          '"1 (25 oz.) Package Frozen Roll Dough, Thawed", "1 '
                          '(3.4 oz.) Package Butterscotch instant Pudding '
                          'Mix", "1/2 Cup Butter or Margarine, Melted", "1/2 '
                          'Cup Firmly Packed Brown Sugar", "3/4 tsp. Ground '
                          'Cinnamon"]\n'
                          '\n'
                          'Generic ingredients: ',
               'role': 'user'},
              {'content': '["Pecans", "Butter", "Brown Sugar", "Ground '
                          'Cinnamon"]',
      

In [16]:
training_data = training.apply(prepare_conversation, axis=1).tolist()
training_set = training_data[:100]
validation_set = training_data[101:200]
training_set[:2]

[{'messages': [{'role': 'system',
    'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
   {'role': 'user',
    'content': 'Title: Christmas Morning Sticky Buns\n\nIngredients: ["1/2 Cup Chopped Pecans or Walnuts", "1 (25 oz.) Package Frozen Roll Dough, Thawed", "1 (3.4 oz.) Package Butterscotch instant Pudding Mix", "1/2 Cup Butter or Margarine, Melted", "1/2 Cup Firmly Packed Brown Sugar", "3/4 tsp. Ground Cinnamon"]\n\nGeneric ingredients: '},
   {'role': 'assistant',
    'content': '["Pecans", "Butter", "Brown Sugar", "Ground Cinnamon"]'}]},
 {'messages': [{'role': 'system',
    'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
   {'role': 'user',
    'content': 'Title: Taco Dip\n\nIngredients: ["1 lb. ground beef", "1 (8 oz.) sour cream", "1 (8 oz.) Philadelphia cream cheese, softened", "1 (12 oz.) shredded Cheddar", "1 packe

In [21]:
def json_file(data, file_name):
    with open(file_name,"w") as tmp:
        for obj in data:
            jobj = json.dumps(obj) + "\n"
            tmp.write(jobj)
    tmp.close()
json_file(training_set,'train.jsonl')
json_file(validation_set,'valid.jsonl')

In [28]:
with open('train.jsonl','rb') as training_json:
    train_response = client.files.create(
        file = training_json, purpose = "fine-tune"
    )
with open('valid.jsonl', "rb") as validation_json:
    validation_response = client.files.create(
        file=validation_json, purpose="fine-tune"
    )
valid_id = validation_response.id
train_id = train_response.id
print(f'train_id: {train_id}')
print(f'valid_id: {valid_id}')

train_id: file-HQxTKUanenM8lUBUeOmznPZq
valid_id: file-1b1d96p2nwZu5idUXbUI2qDA


In [29]:
response = client.fine_tuning.jobs.create(
    training_file=train_id,
    validation_file=valid_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-Vhckx8QKq9r54gyh3GaVlEyW
Status: validating_files


In [38]:
response = client.fine_tuning.jobs.retrieve(job_id)
print(response.status)


running


In [45]:
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

RuntimeError: Fine-tuned model ID not found. Your job has likely not been completed yet.

In [44]:
def prepare_test(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    return  messages

pprint(prepare_test(testing.iloc[0]))

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'Title: Peanut Butter Balls\n'
             '\n'
             'Ingredients: ["2 sticks melted oleo", "1 1/2 c. crunchy peanut '
             'butter", "1 tsp. vanilla", "1 box confectioners sugar", "1 1/2 '
             'c. graham cracker crumbs"]\n'
             '\n'
             'Generic ingredients: ',
  'role': 'user'}]


In [43]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=prepare_test(testing.iloc[0]), temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': 'you must provide a model parameter', 'type': 'invalid_request_error', 'param': None, 'code': None}}